In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Cargar el dataset
#input_file = "reviews_proc.csv"
input_file = "reviews_prod_20_29.csv"
data = pd.read_csv(input_file, sep=';', encoding="utf-8")

# Suponiendo que las reseñas están en la columna 'review'
reseñas = data['review'].astype(str).fillna("")

# Vectorización BoW
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(reseñas)

# Convertir la matriz BoW en un DataFrame
bow_df = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Agregar las estrellas al DataFrame
bow_df['stars'] = data['stars']

# Guardar en un nuevo CSV
bow_df.to_csv("reviews_bow.csv", index=False, sep=';')

print("Archivo reviews_bow.csv generado con éxito.")

Archivo reviews_bow.csv generado con éxito.


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_bow, data['stars'], test_size=0.2, random_state=42)

# Entrenar modelo Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.4f}")

Precisión del modelo: 0.6250
